In [ ]:
# Konstantinos Kyriakou 3015
# mlteam93@cse-uoi

import numpy as np
import pandas as pd
import seaborn as sns 
import zipfile
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Flatten,Input

from sklearn.svm import SVC

f1_scores = list()
acc_scores = list()


In [ ]:
# Train data
train_df_zip = zipfile.ZipFile('../input/ghouls-goblins-and-ghosts-boo/train.csv.zip')
train_df = pd.read_csv(train_df_zip.open('train.csv'))#, header=None)

# Test data
test_df_zip = zipfile.ZipFile('../input/ghouls-goblins-and-ghosts-boo/test.csv.zip')
test_df = pd.read_csv(test_df_zip.open('test.csv'))#, header=None)

train_df

FileNotFoundError: ignored

In [ ]:
# How many Ghouls,Goblins and ghosts are in train data
sns.countplot(x='type',data=train_df)

In [ ]:
# Do colors matter?
sns.countplot(x='type', hue='color', data=train_df, palette=['grey', 'green', 'black', 'lightgrey', 'blue', 'red'])

In [ ]:
# Colors don't matter
train_df.drop('color',inplace  = True,axis = 1)
test_df.drop('color',inplace  = True,axis = 1)

train_df.head()

In [ ]:
# Correlation between train characteristics and monster type
sns.pairplot(train_df.drop(['id'], axis=1), hue='type')

In [ ]:
y = train_df['type']
ySVM = train_df['type'] #needed for svm later

test_ids = test_df['id']

train_df = train_df.drop(['id','type'], axis=1)
test_df = test_df.drop(['id'], axis=1)
train_df.head()

In [ ]:
# kNN method

k = 11

X_train, X_test, y_train, y_test = train_test_split(train_df, y, test_size=0.3, random_state=0)

knn = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
knn.fit(X_train,y_train)
y_pred_knn= knn.predict(X_test)
f1 = metrics.f1_score(y_test, y_pred_knn, average='weighted')
accuracy = metrics.accuracy_score(y_test, y_pred_knn)

y_pred_knn= knn.predict(test_df)

print("> kNN (k="+str(k)+"):")
print("> F1 score: "+str(f1))
print("> Accuracy score: "+str(accuracy))

f1_scores.append(f1)
acc_scores.append(accuracy)

In [ ]:
y_pred_knn = knn.predict(test_df)

Y_knn = pd.DataFrame()
Y_knn['id'] = test_ids
Y_knn['type'] = y_pred_knn
if(accuracy>=max(acc_scores)):
    best_prediction = Y_knn
    best_name = "kNN (k="+str(k)+")"
Y_knn.head()

In [ ]:
# Dummify data for Neural networks
y_backup = y
y = pd.get_dummies(y)
y.head()

In [ ]:
# Neural network #1 

K=200
e=40


X_train,X_test,y_train,y_test = train_test_split(train_df,y,test_size = 0.3,random_state = 0)
model = Sequential()
model.add(Dense(64, input_shape = (train_df.shape[1],))) 
model.add(Dense(K, activation='sigmoid')) # 1st hidden layer
model.add(Dense(3, activation='softmax')) # output layer
model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


train=model.fit(X_train,y_train,epochs=e)

# Un-dummify y_test values
new_y_test = y_test.idxmax(axis=1)

# Predict on X_test
temp_y_nn1_pred = model.predict(X_test)

# Convert prediction matrix
y_nn1_pred =[np.argmax(i) for i in temp_y_nn1_pred]

for i in range(len(y_nn1_pred)):
    if (y_nn1_pred[i]==0):
        y_nn1_pred[i] = 'Ghost'
    elif (y_nn1_pred[i]==1):
        y_nn1_pred[i] = 'Ghoul'
    elif (y_nn1_pred[i]==2):
        y_nn1_pred[i] = 'Goblin'

# Metrics
f1_nn1 = metrics.f1_score(new_y_test, y_nn1_pred, average='weighted')
accuracy_nn1 = metrics.accuracy_score(new_y_test, y_nn1_pred)
print()
print("> Neural network #1:      K="+str(K)+" ,"+str(e)+" epochs")
print("> F1 score: "+str(f1_nn1))
print("> Accuracy score: "+str(accuracy_nn1))
f1_scores.append(f1_nn1)
acc_scores.append(accuracy_nn1)


In [ ]:
tmp_pred = model.predict(test_df)
nn1_pred=[np.argmax(i) for i in tmp_pred]
for i in range(len(nn1_pred)):
    if (nn1_pred[i]==0):
        nn1_pred[i] = 'Ghost'
    elif (nn1_pred[i]==1):
        nn1_pred[i] = 'Ghoul'
    elif (nn1_pred[i]==2):
        nn1_pred[i] = 'Goblin'
nn1_pred[:10]

In [ ]:
Y_nn1 = pd.DataFrame()
Y_nn1['id'] = test_ids
Y_nn1['type'] = nn1_pred
if(accuracy_nn1>=max(acc_scores)):
    best_prediction = Y_nn1
    best_name = "Neural Net. #1 (K="+str(K)+", "+str(K)+" epochs)"

Y_nn1.head()

In [ ]:
# Neutral network #2

(K1, K2) = (50, 25)
e = 40

model2 = Sequential()
model2.add(Dense(64, input_shape = (train_df.shape[1],))) 
model2.add(Dense(K1, activation='sigmoid')) # 1st hidden layer
model2.add(Dense(K2, activation='sigmoid')) # 2nd hidden layer
model2.add(Dense(3, activation='softmax')) # output layer
model2.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


train=model2.fit(X_train,y_train,epochs=e)

# Un-dummify y_test values
new_y_test = y_test.idxmax(axis=1)

# Predict on X_test
temp_y_nn2_pred = model2.predict(X_test)

# Convert prediction matrix
y_nn2_pred =[np.argmax(i) for i in temp_y_nn2_pred]

for i in range(len(y_nn1_pred)):
    if (y_nn2_pred[i]==0):
        y_nn2_pred[i] = 'Ghost'
    elif (y_nn2_pred[i]==1):
        y_nn2_pred[i] = 'Ghoul'
    elif (y_nn2_pred[i]==2):
        y_nn2_pred[i] = 'Goblin'

# Metrics
f1_nn2 = metrics.f1_score(new_y_test, y_nn2_pred, average='weighted')
accuracy_nn2 = metrics.accuracy_score(new_y_test, y_nn2_pred)
print()
print("> Neural network #2:      (K1,K2)=("+str(K1)+","+str(K2)+") ,"+str(e)+" epochs")
print("> F1 score: "+str(f1_nn2))
print("> Accuracy score: "+str(accuracy_nn2))
f1_scores.append(f1_nn2)
acc_scores.append(accuracy_nn2)


In [ ]:
tmp_pred = model2.predict(test_df)

nn2_pred=[np.argmax(i) for i in tmp_pred]
for i in range(len(nn2_pred)):
    if (nn2_pred[i]==0):
        nn2_pred[i] = 'Ghost'
    elif (nn2_pred[i]==1):
        nn2_pred[i] = 'Ghoul'
    elif (nn2_pred[i]==2):
        nn2_pred[i] = 'Goblin'
        
Y_nn2 = pd.DataFrame()
Y_nn2['id'] = test_ids
Y_nn2['type'] = nn2_pred
if(accuracy_nn1>=max(acc_scores)):
    best_prediction = Y_nn2
    best_name = "Neural Net. #1 (K1="+str(K1)+", K2="+str(K2)+", "+str(e)+" epochs)"

Y_nn2.head()

In [ ]:
# SVM linear
y = ySVM
X_train, X_test, y_train, y_test = train_test_split(train_df, y, test_size=0.3, random_state=0)
svm_linear =  SVC(kernel="linear")
svm_linear.fit(X_train, y_train)
y_pred_svm_linear = svm_linear.predict(X_test)
svm_linear_f1 = metrics.f1_score(y_test, y_pred_svm_linear, average= "weighted")
svm_linear_accuracy = metrics.accuracy_score(y_test, y_pred_svm_linear)

print("> SVM (linear):")
print("> F1 score: "+str(svm_linear_f1))
print("> Accuracy score: "+str(svm_linear_accuracy))
f1_scores.append(svm_linear_f1)
acc_scores.append(svm_linear_accuracy)

In [ ]:
y_pred_svm_linear = svm_linear.predict(test_df)
Y_svm_linear = pd.DataFrame()
Y_svm_linear['id'] = test_ids
Y_svm_linear['type'] = y_pred_svm_linear
if(svm_linear_accuracy>=max(acc_scores)):
    best_prediction = Y_svm_linear
    best_name = "SVM Linear"

Y_svm_linear.head()

In [ ]:
# SVM gaussian

svm_gaussian =  SVC(kernel="rbf", gamma="scale")
svm_gaussian.fit(X_train, y_train)
y_pred_svm_gaussian = svm_gaussian.predict(X_test)
svm_gaussian_f1 = metrics.f1_score(y_test, y_pred_svm_gaussian, average= "weighted")
svm_gaussian_accuracy = metrics.accuracy_score(y_test, y_pred_svm_gaussian)

print("> SVM (gaussian):")
print("> F1 score: "+str(svm_gaussian_f1 ))
print("> Accuracy score: "+str(svm_gaussian_accuracy))
f1_scores.append(svm_gaussian_f1)
acc_scores.append(svm_gaussian_accuracy)

In [ ]:

y_pred_svm_gaussian = svm_gaussian.predict(test_df)
Y_svm_gaussian = pd.DataFrame()
Y_svm_gaussian['id'] = test_ids
Y_svm_gaussian['type'] = y_pred_svm_gaussian
if(svm_gaussian_accuracy>=max(acc_scores)):
    best_prediction = Y_svm_gaussian
    best_name = "SVM Gaussian"

Y_svm_gaussian.head()


In [ ]:

print("Best method:")
print(best_name)
print(" accuracy:"+str(max(acc_scores)))
submission = best_prediction
submission


In [ ]:
submission.to_csv("submission.csv",index=False)